In [11]:
import sys

import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt

# See versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Kerkas Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")

# Check for TensorFlow GPU access
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is ", "available" if gpu else "NOT AVAIALBLE")
print(tf.config.list_physical_devices())

Tensor Flow Version: 2.8.0
Kerkas Version: 2.8.0

Python 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]
Scikit-Learn 1.0.2
GPU is  available
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.17304026, -0.04146215,  0.10433082, -0.36331773, -0.45270163,
         0.01394668,  0.12811029,  0.24022567, -0.6851184 , -0.3476092 ]],
      dtype=float32)

In [6]:
tf.nn.softmax(predictions).numpy()
# It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network.
# While this can make the model output more directly interpretable, 
# this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation
# for all models when using a softmax output.

array([[0.09477872, 0.10810717, 0.12507531, 0.07835639, 0.07165648,
        0.11426633, 0.12808518, 0.14328146, 0.05679605, 0.07959697]],
      dtype=float32)

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.1692233

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [17]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="accuracy",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=3,
        verbose=1,
    )
]

In [18]:
model.fit(
    x_train, 
    y_train, 
    epochs=20,
    callbacks=callbacks,
    validation_split=0.2
    
)

Epoch 1/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0388 - accuracy: 0.9875 - val_loss: 0.0469 - val_accuracy: 0.9853
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0351 - accuracy: 0.9885 - val_loss: 0.0536 - val_accuracy: 0.9827
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0342 - accuracy: 0.9884 - val_loss: 0.0581 - val_accuracy: 0.9832
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0298 - accuracy: 0.9901 - val_loss: 0.0554 - val_accuracy: 0.9849
Epoch 5/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0262 - accuracy: 0.9910 - val_loss: 0.0594 - val_accuracy: 0.9822
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0264 - accuracy: 0.9912 - val_loss: 0.0560 - val_accuracy: 0.9839
Epoch 7/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0241 - accuracy: 0.9917 - val_loss: 0.0534 - val_accuracy:

In [19]:
model.evaluate(x_test,  y_test, verbose=2)

2022-03-20 20:44:32.150339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 - 1s - loss: 0.0880 - accuracy: 0.9804 - 894ms/epoch - 3ms/step


[0.08796381950378418, 0.980400025844574]

In [20]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [21]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.2004799e-14, 1.4932239e-14, 2.5587585e-10, 3.7690168e-07,
        1.3603591e-19, 1.2230213e-13, 4.3756356e-21, 9.9999964e-01,
        9.1751595e-13, 2.7342537e-09],
       [2.8115333e-20, 4.6236637e-10, 1.0000000e+00, 8.5046512e-15,
        2.3848027e-28, 3.9528783e-16, 5.2824188e-19, 8.5493540e-26,
        3.9238191e-15, 1.5467295e-28],
       [6.7147290e-12, 9.9997532e-01, 3.3595036e-06, 1.3989714e-08,
        6.0108036e-08, 1.3805742e-11, 9.0426377e-09, 1.6558855e-06,
        1.9525631e-05, 5.2123448e-11],
       [1.0000000e+00, 7.0407287e-20, 7.0787398e-09, 1.0939377e-11,
        1.9954743e-14, 3.1143931e-12, 2.5756344e-09, 7.0243620e-12,
        8.9033205e-14, 1.4286011e-10],
       [1.7995109e-09, 6.1944757e-17, 3.3476373e-09, 3.2274181e-13,
        9.9997723e-01, 8.7726414e-13, 3.8382915e-11, 8.2459621e-08,
        4.9505958e-11, 2.2707010e-05]], dtype=float32)>